## Summary of Study Results

With the data downloads from Replica, create a single table with a row for each origin station and columns for “car trips to areas around other destination stations on the other lines” and another column for “train trips”

In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import numpy as np

from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis import utils
from calitp_data_analysis import get_fs
import gcsfs as fs
fs = get_fs()

In [2]:
import replica_utils

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/MetroLink_LinkUS/"


In [5]:
# analysis_result_names_subset_test = ["sbline_origin_covina", "sbline_origin_calstate_la", 
#                                 "riversideline_origin_industry",  "riversideline_origin_montebello"]

In [6]:
all_stations_list = ["lossan-simi-valley", "lossan-burbank-airport", "lossan-oxnard",  "lossan-camarillo", "lossan-moorpark",  "lossan-chatsworth",
                     "lossan-northridge", "lossan-van-nuys", "lossan-burbank", "lossan-glendale", 
                     "lossan-commerce", "lossan-norwalk", "lossan-buena-park", "lossan-fullerton", "lossan-anaheim",
                     "lossan-orange", "lossan-santa-ana", "lossan-tustin", "lossan-irvine", "lossan-laguna-niguel", 
                     "lossan-san-juan-capistrano", "lossan-san-clemente", "lossan-san-clemente-pier", "lossan-oceanside", 
                     "lossan-solana-beach", "lossan-san-diego", "lossan-ot-san-diego", "lossan-union-station",
                     
                     "riverside-line-montebello-commerce", "riverside-line-industry", "riverside-line-pomona",
                     "riverside-line-ontario","riverside-line-pedley",  "riverside-line-riverside",
                     
                     "sb-line-cal-state-la", "sb-line-el-monte",  "sb-line-baldwin-park", "sb-line-covina",  "sb-line-pomona-north",
                     "sb-line-claremont", "sb-line-montclair", "sb-line-upland", "sb-line-rancho-cucamonga", "sb-line-fontana", 
                     "sb-line-rialto", "sb-line-san-bernardino-depot", "sb-line-san-bernardino-dt", 
                     "sb-line-redlands-dt", "sb-line-redlands-esri", "sb-line-redlands-univ", "sb-line-san-bernardino-tippecanoe",
                     
                     "91-perris-line-corona", "91-perris-line-corona-west", "91-perris-line-moreno-valley-march", 
                     "91-perris-line-perris-south", "91-perris-line-perris-downtown", "91-perris-line-placentia-future", 
                     "91-perris-line-riverside-hunter-park", "91-perris-line-riverside-la-sierra",
                     
                     "antelope-valley-burbank-airport-n", "antelope-valley-lancaster", "antelope-valley-newhall", "antelope-valley-palmdale",
                     "antelope-valley-santa-clarita", "antelope-valley-sun-valley", "antelope-valley-sylmar-san-fernando",
                     "antelope-valley-via-princessa", "antelope-valley-vincent-grade", "antelope-valley-vista-canyon",
                     
                     "inland-empire-oc-anaheim-canyon", 
                     
                     "ventura-line-ventura-east", "ventura-line-ventura-dtwn",                    
                    ]   

In [7]:
gdf_stations = replica_utils.read_in_stations(all_stations_list)

In [8]:
gdf_stations.sample()

,geoid,geoname,alias,centrlat,centrlon,geometry,station_name,station_name_full,transit_lines
83,3604677,"1106.03 (Orange, CA)",None,33.8686,-117.9934,"POLYGON ((-118.00960 33.87338, -118.00936 33.8...",lossan-buena-park,Buena Park,"Orange County Line, 91/Perris Valley Line"


In [9]:
analysis_result_names = [ "line_origin_union_station",
                         "riversideline_origin_montebello", "riversideline_origin_industry", "riversideline_origin_pomona",  
                         "riversideline_origin_ontario", "riversideline_origin_pedley",  "riversideline_origin_riverside",
                         
                         "sbline_origin_calstate_la", "sbline_origin_el_monte",  "sbline_origin_baldwin_park",
                         "sbline_origin_covina",  "sbline_origin_pomona_north", "sbline_origin_fontana",
                         "sbline_origin_claremont", "sbline_origin_montclair", "sbline_origin_upland", "sbline_origin_rancho_cucamonga", 
                         "sbline_origin_rialto", "sbline_origin_san_bernardino_depot", "sbline_origin_san_bernardino_dt", 
                         
                         "origin_simi_valley", "origin_burbank_airport", "origin_oxnard",  "origin_camarillo", "origin_moorpark",  "origin_chatsworth",
                         "origin_northridge", "origin_van_nuys", "origin_burbank", "origin_glendale", 
                         "origin_commerce", "origin_norwalk", "origin_buena_park", "origin_fullerton", "origin_anaheim",
                         "origin_orange", "origin_santa_ana", "origin_tustin", "origin_irvine", "origin_laguna_niguel", 
                         "origin_san_juan_capistrano", "origin_san_clemente", "origin_san_clemente_pier", "origin_oceanside", 
                         "origin_solana_beach", "origin_san_diego", "origin_old_town_san_diego",

                         "origin_corona", "origin_corona_west", "origin_moreno_valley", 
                         "origin_perris_south", "origin_perris", "origin_placentia", 
                         "origin_riverside_hunter_park", #"origin_riverside_la_sierra",

                         "origin_burbank_airport_north", "origin_lancaster", "origin_newhall", "origin_palmdale",
                         "origin_santa_clarita", "origin_sun_valley", "origin_sylmar",
                         "origin_via_princessa", "origin_vincent_grade", "origin_vista_canyon",

                         "origin_anaheim_canyon", 

                         "origin_ventura_east", "origin_ventura_downtown_beach",        
                         
                        ]

In [23]:
results = []
for analysis in analysis_result_names:
    with get_fs().open(f"{gcs_path}replica_data_downloads/replica-socal_travel_analysis_{analysis}-trips_dataset.csv") as f:
        
        df = to_snakecase(pd.read_csv(f, dtype={25: str, 26:str}, low_memory=False))
                
        station_col_map = dict(zip(gdf_stations['geoname'], gdf_stations['station_name_full']))
        station_line_map = dict(zip(gdf_stations['station_name_full'], gdf_stations['transit_lines']))
        
        df['origin_tract_station_area'] = df['origin_trct_2020'].map(station_col_map)
        df['destination_tract_station_area'] = df['destination_trct_2020'].map(station_col_map)
        
        station = analysis.split('origin_')[1]  # Get the part after '_origin_'
        station_name = station.replace('_', ' ').upper()
        # line = gdf_stations
        
        auto_df = (df[df.primary_mode=="private_auto"])
        transit_df = (df[df.primary_mode=="public_transit"])
        
        all_trip_count = len(df)
        
        # metrolink_line = replica_utils.define_transit_line(line) 
        n_total_trips = len(df)
        n_private_auto_trips = len(auto_df)
        pct_private_auto_trips = ((len(auto_df)) / (len(df)))
        n_public_transit_trips = (len(transit_df))
        pct_public_transit_trips = ((len(transit_df)) / (len(df)))
        
        auto_mean_min, auto_median_min, auto_mean_miles, auto_median_miles, auto_max_min, auto_max_miles = replica_utils.calc_auto_travel_info(df)
        transit_mean_min, transit_median_min, transit_mean_miles, transit_median_miles, transit_max_miles, transit_max_min = replica_utils.calc_transit_travel_info(df)
        
        auto_top_name1, = replica_utils.get_top_and_bottom_tract_counts(auto_df, top_least = "top", all_trips=all_trip_count)
        auto_bottom_name1,  = replica_utils.get_top_and_bottom_tract_counts(auto_df, top_least = "least", all_trips=all_trip_count)
        # auto_top_name2, auto_top_name3,  auto_bottom_name2, auto_bottom_name3
        
        transit_top_name1,  = replica_utils.get_top_and_bottom_tract_counts(transit_df, top_least = "top", all_trips=all_trip_count)
        transit_bottom_name1, = replica_utils.get_top_and_bottom_tract_counts(transit_df, top_least = "least", all_trips=all_trip_count)
         # transit_top_name2, transit_top_name3
             # transit_bottom_name2, transit_bottom_name3 
        
        ## set up the table for all the results
        results.append({
                'station_name': station_name,
                # 'metrolink_line': metrolink_line,
                'total_trips_from_origin_station': n_total_trips,
                'n_auto_trips_to_other_station_areas': n_private_auto_trips,
                'pct_auto_trips_to_other_station_areas': pct_private_auto_trips,
                'n_tranist_trips_to_other_station_areas': n_public_transit_trips,
                'pct_transit_trips_to_other_station_areas': pct_public_transit_trips,
                'auto_mean_minutes': auto_mean_min,
                'auto_median_minutes': auto_median_min,
                'auto_max_minutes': auto_max_min, 
                'auto_mean_miles': auto_mean_miles,
                'auto_median_miles': auto_median_miles,
                'auto_max_miles': auto_max_miles,
                'transit_mean_minutes': transit_mean_min,
                'transit_median_minutes': transit_median_min,
                'transit_max_minutes': transit_max_min,
                'transit_mean_miles': transit_mean_miles,
                'transit_median_miles': transit_median_miles,
                'transit_max_miles':transit_max_miles,
            
                'auto_top_tract_traveled_to': auto_top_name1,
                # 'auto_top_tract_traveled_to_pct':auto_top_num1, 
                # 'auto_second_top_tract_traveled_to':auto_top_name2, 
                # 'auto_second_top_tract_traveled_to_pct':auto_top_num2, 
                # 'auto_third_top_tract_traveled_to':auto_top_name3, 
                # 'auto_third_top_tract_traveled_to_pct':auto_top_num3, 
                
                'auto_least_tract_traveled_to': auto_bottom_name1,
                # 'auto_second_least_tract_traveled_to':auto_bottom_name2, 
                # 'auto_third_least_tract_traveled_to':auto_bottom_name3,  
                
                'transit_top_tract_traveled_to': transit_top_name1,
#                 # 'transit_top_tract_traveled_to_pct':transit_top_num1, 
                # 'transit_second_top_tract_traveled_to':transit_top_name2, 
#                 # 'transit_second_top_tract_traveled_to_pct':transit_top_num2, 
                # 'transit_third_top_tract_traveled_to':transit_top_name3, 
#                 # 'atransit_third_top_tract_traveled_to_pct':transit_top_num3, 
                
                'transit_least_tract_traveled_to': transit_bottom_name1,
                # 'transit_second_least_tract_traveled_to':transit_bottom_name2, 
                # 'transit_third_least_tract_traveled_to':transit_bottom_name3
                })

result_summary = pd.DataFrame(results)        
       

In [24]:
result_summary

,station_name,total_trips_from_origin_station,n_auto_trips_to_other_station_areas,pct_auto_trips_to_other_station_areas,n_tranist_trips_to_other_station_areas,pct_transit_trips_to_other_station_areas,auto_mean_minutes,auto_median_minutes,auto_max_minutes,auto_mean_miles,auto_median_miles,auto_max_miles,transit_mean_minutes,transit_median_minutes,transit_max_minutes,transit_mean_miles,transit_median_miles,transit_max_miles,auto_top_tract_traveled_to,auto_least_tract_traveled_to,transit_top_tract_traveled_to,transit_least_tract_traveled_to
0,UNION STATION,18657,10339,0.554162,938,0.050276,31.680143,24.0,179,21.165722,14.3,123.8,78.638593,64.0,1434,21.760768,21.10,74.2,Cal State LA,Solana Beach,Glendale,Perris - Downtown
1,MONTEBELLO,42917,22560,0.525666,655,0.015262,13.369238,7.0,145,7.194459,2.7,117.5,42.964885,30.0,217,7.341221,3.50,84.0,Montebello/Commerce,Redlands - Esri,Montebello/Commerce,San Bernardino - Downtown
2,INDUSTRY,8588,5436,0.632976,51,0.005939,26.223142,23.0,128,17.304213,13.9,112.2,97.647059,82.0,211,26.864706,23.90,66.8,Covina,San Diego,LA Union Station,Commerce
3,POMONA,19454,10982,0.564511,186,0.009561,15.845110,11.0,107,9.885230,5.4,98.3,58.634409,47.0,237,13.086559,10.60,89.5,Montclair,Simi Valley,Covina,Redlands - University
4,ONTARIO,63621,35456,0.557300,74,0.001163,18.636084,16.0,140,11.136967,9.1,124.2,84.351351,82.5,187,17.785135,16.00,47.7,Fontana,Via Princessa,Montclair,San Bernardino - Tippecanoe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,VINCENT GRADE,6609,3532,0.534423,1,0.000151,22.870895,18.0,186,17.500651,14.0,100.6,45.000000,45.0,45,10.000000,10.00,10.0,Palmdale,Burbank Airport,Palmdale,Palmdale
63,VISTA CANYON,38595,24929,0.645913,218,0.005648,9.828393,7.0,143,6.172321,3.5,102.1,36.894495,26.0,203,7.613303,2.85,80.8,Via Princessa,Corona - North Main,Via Princessa,Glendale
64,ANAHEIM CANYON,46536,26235,0.563757,208,0.004470,12.597103,10.0,140,7.972346,5.4,116.4,61.225962,47.0,207,11.703365,7.05,70.5,Placentia,Camarillo,Placentia,Burbank - Downtown
65,VENTURA EAST,35910,22712,0.632470,261,0.007268,13.641555,10.0,254,9.180653,5.8,233.4,42.015326,37.0,129,5.478927,5.10,18.3,Ventura - Downtown / Beach,Anaheim Canyon,Oxnard,Ventura - Downtown / Beach


In [26]:
## export
# result_summary.to_csv("MetroLink_Stations_Replica_Summaries_072225.csv")

In [13]:
# df_auto.columns

In [14]:
# df.primary_mode.value_counts()

In [15]:
#### add in which station area is most traveled to

In [16]:
## starting with one station
with get_fs().open(f"{gcs_path}replica_data_downloads/replica-socal_travel_analysis_sbline_origin_covina-trips_dataset.csv") as f:
        df = to_snakecase(pd.read_csv(f, dtype={25: str, 26:str}))

In [17]:
df.sample()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
8958,11692922982317450563,"3 (Tract 4037.21, Los Angeles, CA)","4037.21 (Los Angeles, CA)","Los Angeles County, CA",California,"2 (Tract 4052.03, Los Angeles, CA)","4052.03 (Los Angeles, CA)","Los Angeles County, CA",California,private_auto,shop,home,08:56:00,09:06:25,10,5.0,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,single_family,single_family,retail,retail,3362481888867496757,9724064364509890659,72.0,female,hispanic_or_latino_origin,not_in_labor_force,unemployed_under_16_not_in_labor_force,17994.0,other_travel_mode,5.0,410501.0,three_plus,core,not_working,single_family,not_attending_school,high_school,owner,spanish,"3 (Tract 4037.21, Los Angeles, CA)","4037.21 (Los Angeles, CA)","Los Angeles County, CA",California,Does not have work/school location,Does not have work/school location,Does not have work/school location,Does not have work/school location


In [18]:
### mapping the stations names to the census tracts

In [19]:
all_stations_list = ["lossan-simi-valley", "lossan-burbank-airport", "lossan-oxnard",  "lossan-camarillo", "lossan-moorpark",  "lossan-chatsworth",
                     "lossan-northridge", "lossan-van-nuys", "lossan-burbank", "lossan-glendale", 
                     "lossan-commerce", "lossan-norwalk", "lossan-buena-park", "lossan-fullerton", "lossan-anaheim",
                     "lossan-orange", "lossan-santa-ana", "lossan-tustin", "lossan-irvine", "lossan-laguna-niguel", 
                     "lossan-san-juan-capistrano", "lossan-san-clemente", "lossan-san-clemente-pier", "lossan-oceanside", 
                     "lossan-solana-beach", "lossan-san-diego", "lossan-ot-san-diego", "lossan-union-station",
                     
                     "riverside-line-montebello-commerce", "riverside-line-industry", "riverside-line-pomona",
                     "riverside-line-ontario","riverside-line-pedley",  "riverside-line-riverside",
                     
                     "sb-line-cal-state-la", "sb-line-el-monte",  "sb-line-baldwin-park", "sb-line-covina",  "sb-line-pomona-north",
                     "sb-line-claremont", "sb-line-montclair", "sb-line-upland", "sb-line-rancho-cucamonga", "sb-line-fontana", 
                     "sb-line-rialto", "sb-line-san-bernardino-depot", "sb-line-san-bernardino-dt", 
                     "sb-line-redlands-dt", "sb-line-redlands-esri", "sb-line-redlands-univ", "sb-line-san-bernardino-tippecanoe",
                     
                     "91-perris-line-corona", "91-perris-line-corona-west", "91-perris-line-moreno-valley-march", 
                     "91-perris-line-perris-south", "91-perris-line-perris-downtown", "91-perris-line-placentia-future", 
                     "91-perris-line-riverside-hunter-park", "91-perris-line-riverside-la-sierra",
                     
                     "antelope-valley-burbank-airport-n", "antelope-valley-lancaster", "antelope-valley-newhall", "antelope-valley-palmdale",
                     "antelope-valley-santa-clarita", "antelope-valley-sun-valley", "antelope-valley-sylmar-san-fernando",
                     "antelope-valley-via-princessa", "antelope-valley-vincent-grade", "antelope-valley-vista-canyon",
                     
                     "inland-empire-oc-anaheim-canyon", 
                     
                     "ventura-line-ventura-east", "ventura-line-ventura-dtwn",                    
                    ]   

In [20]:
test_all_stations.sample()

NameError: name 'test_all_stations' is not defined

In [ ]:
df.destination_tract_station_area.value_counts()